In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

# Generate all interactions

In [10]:
#source https://stackoverflow.com/questions/43106707/generate-all-possible-2-character-combinations-of-a-potential-8-character-string
from itertools import product, combinations

def build(num_chars, length):
    bases = ["U", "C", "A", "G"]
    for letters in product(bases, repeat=num_chars):
        for positions in combinations(range(length), num_chars):
            yield tuple(zip(letters, positions))

In [11]:

pre_mono = list(build(1,8))
post_mono = []
for i in range(len(pre_mono)):
    post_mono.append((pre_mono[i][0],(False,False)))
    
dint_combos = list(build(2,8))
complete_combinations = post_mono+dint_combos

In [21]:
complete_combinations

[(('U', 0), (False, False)),
 (('U', 1), (False, False)),
 (('U', 2), (False, False)),
 (('U', 3), (False, False)),
 (('U', 4), (False, False)),
 (('U', 5), (False, False)),
 (('U', 6), (False, False)),
 (('U', 7), (False, False)),
 (('C', 0), (False, False)),
 (('C', 1), (False, False)),
 (('C', 2), (False, False)),
 (('C', 3), (False, False)),
 (('C', 4), (False, False)),
 (('C', 5), (False, False)),
 (('C', 6), (False, False)),
 (('C', 7), (False, False)),
 (('A', 0), (False, False)),
 (('A', 1), (False, False)),
 (('A', 2), (False, False)),
 (('A', 3), (False, False)),
 (('A', 4), (False, False)),
 (('A', 5), (False, False)),
 (('A', 6), (False, False)),
 (('A', 7), (False, False)),
 (('G', 0), (False, False)),
 (('G', 1), (False, False)),
 (('G', 2), (False, False)),
 (('G', 3), (False, False)),
 (('G', 4), (False, False)),
 (('G', 5), (False, False)),
 (('G', 6), (False, False)),
 (('G', 7), (False, False)),
 (('U', 0), ('U', 1)),
 (('U', 0), ('U', 2)),
 (('U', 0), ('U', 3)),
 ((

# Get interactions of each seq

In [4]:
predicted_eff = pd.read_csv("dinucleotide_TIS_eff.csv")

In [5]:
del predicted_eff['lower.bound']
del predicted_eff['upper.bound']

In [6]:
def make_keys(sequence):
    interactions = []
    
    #remove AUG
    stripped_TIS = sequence[:6]+sequence[9:]
    range_2 = list(range(len(stripped_TIS)))

    #make the mono interactions
    for a in range(len(stripped_TIS)):
        monont_pos_key = ((stripped_TIS[a],a),(False,False))
        interactions.append(monont_pos_key)

    #make the dint interactions
    for i in range(len(stripped_TIS)-1):

        range_2.pop(0)
        for j in range_2:
            dint_pos_key = ((stripped_TIS[i],i),(stripped_TIS[j],j))
            interactions.append(dint_pos_key)
    return interactions

In [7]:
def convert_seq_to_binary(interactions, complete_combinations):
    binary_key = []
    for combination in complete_combinations:
        if combination in interactions:
            binary_key.append(1)
        elif combination not in interactions:
            binary_key.append(0)
    return binary_key

In [8]:
#make a binaries of all seqs in predicted efficienies
binaries_list = []
for each in predicted_eff['sequence']:
    binaries_list.append(convert_seq_to_binary(make_keys(each),complete_combinations))

KeyboardInterrupt: 

In [ ]:
#merge binaries list and efficienies together
to_merge = pd.DataFrame(binaries_list,columns=complete_combinations)

to_merge['efficiency'] = predicted_eff['efficiency']



In [ ]:
to_merge.to_csv("mono_and_di_interactions.csv")

# mono only

In [ ]:
binaries_mono = []
for each in predicted_eff['sequence']:
    binaries_mono.append(convert_seq_to_binary(make_keys(each),post_mono))

In [ ]:
binaries_di =[]
for each in predicted_eff['sequence']:
    binaries_di.append(convert_seq_to_binary(make_keys(each), dint_combos))

In [ ]:
mono_df = pd.DataFrame(binaries_mono, columns = post_mono)
mono_df['efficiency'] = predicted_eff['efficiency']
mono_df.to_csv("mono_interactions.csv")

di_df = pd.DataFrame(binaries_di, columns = dint_combos)
di_df['efficiency'] = predicted_eff['efficiency']
di_df.to_csv("di_interactions.csv")

In [ ]:
mono_df

# Make binary vector of seq compared to all combos

In [ ]:
# [first or second tuple] [first or second value in the tuple]
# [0][0] = 1st value in 1st tuple
# [0][1] = 2nd value in 1st tuple
# 
# [1][0] = 1st value in 2nd tuple
# [1][1] = 2nd value in 2nd tuple

# lookup_key(pwm_key)